# Sleep Analysis
Yay! We made it to the analysis.

For details on how the data was imported, see `import_data.ipynb`.

For details on how the data was cleaned, see `clean_and_explore.ipynb`.  


NOTE: I lost my Fitbit tracker on August 25, 2020, and replaced it November 7, 2020.  
**Therefore, there is a gap in the data between 2020-08-24 and 2020-11-06.**

In [15]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

Merge into one giant table for easier plotting 

#### Read all of the data in the `cleaned_data/` directory. 

In [108]:
sleep_score = pd.read_pickle('cleaned_data/sleep_score.pkl')
sleep_detail = pd.read_pickle('cleaned_data/sleep_detail.pkl')

lightly_active = pd.read_pickle('cleaned_data/lightly_active.pkl')
moderately_active = pd.read_pickle('cleaned_data/moderately_active.pkl')
very_active = pd.read_pickle('cleaned_data/very_active.pkl')
sedentary_minutes = pd.read_pickle('cleaned_data/sedentary_minutes.pkl')

step_daily = pd.read_pickle('cleaned_data/step_daily.pkl')

bpm = pd.read_pickle('cleaned_data/bpm.pkl')
resting_heartrate = pd.read_pickle('cleaned_data/resting_heartrate.pkl')

In [111]:
# A list of all the dataframes 
dfs = [
    sleep_score, 
    sleep_detail, 
    lightly_active,
    moderately_active,
    very_active,
    sedentary_minutes,
    step_daily,
    bpm,
    resting_heartrate,
]

## Merge the data into one big table
Merge the data using an inner join on the index of all the tables (`date`). 

Having all the data in the same table will make plotting faster. 

In [112]:
daily_stats = pd.concat(dfs, join='inner', axis=1)

In [115]:
daily_stats.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 932 entries, 2019-08-22 to 2022-12-19
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall_sleep_score      932 non-null    float64
 1   sleep_resting_heartrate  932 non-null    float64
 2   deep_sleep_in_minutes    932 non-null    int64  
 3   minutesAsleep            932 non-null    int64  
 4   mainSleep                932 non-null    int64  
 5   deep_mins                932 non-null    float64
 6   wake_mins                932 non-null    float64
 7   light_mins               932 non-null    float64
 8   rem_mins                 932 non-null    float64
 9   value                    932 non-null    int64  
 10  value                    932 non-null    int64  
 11  value                    932 non-null    int64  
 12  value                    932 non-null    int64  
 13  daily_steps              932 non-null    int64  
 14  bpm_max

## Do a quick correlation matrix 